In [2]:
import numpy as np
import os
import cv2
from numpy.lib.stride_tricks import sliding_window_view
import warnings
import tensorflow as tf
from tensorflow import keras

CONVOLUTION:

In [3]:
def batch_convolution(data, filters, stride, padding='valid'):
    """
    Optimized batch convolution operation with 4D input support
    """
    # Input validation code remains unchanged
        # Input validation
    if not isinstance(data, np.ndarray) or data.ndim != 4:
        raise ValueError("Input data must be 4D numpy array [B, C, H, W]")
    
    if not isinstance(filters, np.ndarray) or filters.ndim != 4:
        raise ValueError("Filters must be 4D numpy array [N, C, FH, FW]")
    
    if filters.shape[1] != data.shape[1]:
        raise ValueError(f"Filter channels ({filters.shape[1]}) must match data channels ({data.shape[1]})")
    
    batch_size, in_channels, in_h, in_w = data.shape
    num_filters, _, fh, fw = filters.shape
    sh, sw = stride
    batch_size, in_channels, in_h, in_w = data.shape
    num_filters, _, fh, fw = filters.shape
    sh, sw = stride
    
    # Calculate output dimensions remains unchanged
    if padding == 'same':
        i=0
        # Same padding calculation code
        # ...
    elif padding == 'valid':
        out_h = (in_h - fh) // sh + 1
        out_w = (in_w - fw) // sw + 1
    else:
        raise ValueError("Padding must be 'valid' or 'same'")

    # Create sliding windows for each sample in the batch
    windows = sliding_window_view(data, (1, 1, fh, fw), axis=(0, 1, 2, 3))
    
    # Apply stride
    windows = windows[:, :, ::sh, ::sw, 0, :, :, :]
    
    # FIXED: Get actual window shape rather than trying to unpack
    window_shape = windows.shape
    
    # FIXED: Reshape using actual dimensions
    # Flatten batch, channels, and spatial dimensions
    windows_reshaped = windows.reshape(-1, in_channels * fh * fw)
    
    # Reshape filters to match inner dimension
    filters_reshaped = filters.reshape(num_filters, in_channels * fh * fw)
    
    # Matrix multiplication with matching dimensions
    output = np.matmul(windows_reshaped, filters_reshaped.T)
    
    # Reshape output to proper dimensions
    output = output.reshape(batch_size, out_h, out_w, num_filters)
    output = output.transpose(0, 3, 1, 2)  # NHWC -> NCHW
    
    return output


DECONVOLUTION:

In [4]:
def batch_deconvolution(data, filters, stride, padding='valid', output_padding=(0,0)):
    """
    Vectorized transposed convolution (deconvolution) with batch support
    
    Parameters:
    data : np.ndarray [batch_size, in_channels, in_height, in_width]
    filters : np.ndarray [out_channels, in_channels, fh, fw]
    stride : tuple (sh, sw)
    padding : 'valid' or 'same'
    output_padding : tuple (oph, opw)
    
    Returns:
    output : np.ndarray [batch_size, out_channels, out_height, out_width]
    """
    # Input validation
    if data.ndim != 4:
        raise ValueError("Input must be 4D [batch, channels, height, width]")
    if filters.ndim != 4:
        raise ValueError("Filters must be 4D [out_chan, in_chan, fh, fw]")
    
    batch_size, in_chan, in_h, in_w = data.shape
    out_chan, _, fh, fw = filters.shape
    sh, sw = stride
    oph, opw = output_padding

    # Calculate output dimensions
    if padding == 'same':
        out_h = in_h * sh + oph
        out_w = in_w * sw + opw
        pad_h = (in_h - 1) * sh + fh - out_h + oph
        pad_w = (in_w - 1) * sw + fw - out_w + opw
        pad_h = max(pad_h, 0)
        pad_w = max(pad_w, 0)
    elif padding == 'valid':
        out_h = (in_h - 1) * sh + fh + oph
        out_w = (in_w - 1) * sw + fw + opw
        pad_h = pad_w = 0
    else:
        raise ValueError("Padding must be 'valid' or 'same'")
    # Create upsampled input with zeros
    upsampled = np.zeros((batch_size, in_chan, (in_h-1)*sh+1, (in_w-1)*sw+1))
    upsampled[:, :, ::sh, ::sw] = data

    # Apply padding
    pad_top = pad_h // 2
    pad_bottom = pad_h - pad_top
    pad_left = pad_w // 2
    pad_right = pad_w - pad_left
    padded = np.pad(upsampled, [(0,0), (0,0), (pad_top, pad_bottom), (pad_left, pad_right)],mode='constant')

    # Create windows view
    #windows = sliding_window_view(upsampled, (fh, fw), axis=(2,3))
    #windows = windows.reshape(batch_size, in_chan, -1, fh, fw)

    # Adjust einsum for 5D tensor - 's' represents combined spatial dimension
    #output = np.einsum('bisjk,oijk->bos', windows, filters)
    
    # Reshape output to recover separate spatial dimensions
    #spatial_h = (padded.shape[2] - fh + 1)
    #spatial_w = (padded.shape[3] - fw + 1)
    #output = output.reshape(batch_size, out_chan, spatial_h, spatial_w)

    # Perform batched deconvolution using einsum
    #output = np.einsum('bihwpq,oipq->bohw', windows, filters)

    # Apply output padding
    #if any(output_padding):
        #output = np.pad(output,
                       #[(0,0), (0,0), (0, oph), (0, opw)],
                       #mode='constant')

    #return output[:, :, :out_h, :out_w]
    output=np.zeros((32, out_chan, out_h, out_w))
    for i in range(0,32):
        for j in range(0,out_chan):
            for k in range(0,in_h):
                for l in range(0,in_w):
                    current=filters[j,:,:,:]
                    for u in range(0,in_chan):
                        current[u,:,:]*=data[i,u,k,l]
                    output[i,j,k*sh:k*sh+fh,l*sw:l*sw+fw]+=np.sum(current,axis=0)                

    return output

RELU:

In [5]:
def relu(data):
    return np.maximum(0.0,data)

BATCH NORMALIZATION:

In [6]:
def batch_norm(data, bn, eps=1e-5):

    gamma=bn[0,:]
    beta=bn[1,:]
    """  
    data : np.ndarray [N, C, H, W]  
    gamma: np.ndarray [C,]  
    beta : np.ndarray [C,]  
    """  
    # Per-channel statistics  
    mu = np.mean(data, axis=(0, 2, 3), keepdims=True)  
    var = np.var(data, axis=(0, 2, 3), keepdims=True)  

    # Normalization  
    x_hat = (data - mu) / np.sqrt(var + eps)  

    # Scale and shift  
    return gamma[None,:,None,None] * x_hat + beta[None,:,None,None]  

DERIVATIVE FOR BATCH NORMALIZATION:

In [7]:
def batch_norm_backward(dout, x, gamma, eps=1e-5):  
    mu = np.mean(x, axis=(0,2,3), keepdims=True)  
    var = np.var(x, axis=(0,2,3), keepdims=True)  
    x_hat = (x - mu)/np.sqrt(var + eps)  

    N, C, H, W = x.shape  
    dgamma = np.sum(dout * x_hat, axis=(0,2,3))  
    dbeta = np.sum(dout, axis=(0,2,3))  

    dx_hat = dout * gamma[None,:,None,None]  
    dvar = np.sum(dx_hat * (x - mu) * (-0.5) * (var + eps)**(-1.5), axis=(0,2,3))  
    dmu = np.sum(dx_hat * (-1)/np.sqrt(var + eps), axis=(0,2,3)) + dvar * np.mean(-2*(x - mu), axis=(0,2,3))  

    dx = dx_hat / np.sqrt(var + eps) + dvar[None,:,None,None]*2*(x - mu)/N/H/W + dmu[None,:,None,None]/N/H/W  
    dbn=np.zeros((2,gamma.shape[0]))
    dbn[0,:]=dgamma
    dbn[1,:]=dbeta
    return dx, dbn  


DERIVATIVE FOR RELU:

In [8]:
def relu_backward(data):
    return (data>0).astype(float)

DERIVATIVE FOR CONVOLUTION:

In [9]:
def conv_backward(dout, x, W, stride, padding='valid'):
    """
    Vectorized backward pass for batch convolution
    
    Parameters:
    dout : np.ndarray [batch, num_filters, out_h, out_w]
    x    : np.ndarray [batch, in_channels, h_in, w_in]
    W    : np.ndarray [num_filters, in_channels, fh, fw]
    stride, padding: Same as forward pass
    
    Returns:
    dx   : np.ndarray [batch, in_channels, h_in, w_in]
    dW   : np.ndarray [num_filters, in_channels, fh, fw]
    """
    batch_size, in_chan, h_in, w_in = x.shape
    num_filters, _, fh, fw = W.shape
    sh, sw = stride
    
    # Recreate forward pass padding
    if padding == 'same':
        out_h = int(np.ceil(h_in / sh))
        out_w = int(np.ceil(w_in / sw))
        pad_h = max((out_h - 1) * sh + fh - h_in, 0)
        pad_w = max((out_w - 1) * sw + fw - w_in, 0)
        x_padded = np.pad(x, [(0,0), (0,0), 
                           (pad_h//2, pad_h - pad_h//2),
                           (pad_w//2, pad_w - pad_w//2)])
    else:
        x_padded = x
    
    # Get output dimensions from dout
    _, _, out_h, out_w = dout.shape
    
    # Initialize gradients
    dW = np.zeros_like(W)
    dx_padded = np.zeros_like(x_padded)
    
    # Compute dW (filter gradients) without using einsum
    for b in range(batch_size):
        for i in range(in_chan):
            for o in range(num_filters):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * sh
                        w_start = w * sw
                        x_patch = x_padded[b, i, h_start:h_start+fh, w_start:w_start+fw]
                        dW[o, i] += x_patch * dout[b, o, h, w]
    
    # Compute dx (input gradients)
    W_flipped = np.flip(W, (2,3)).transpose(1,0,2,3)  # [in_chan, out_chan, fh, fw]
    
    # Calculate required padding for valid convolution
    pad_h = fh - 1 - (pad_h//2 if padding=='same' else 0)
    pad_w = fw - 1 - (pad_w//2 if padding=='same' else 0)
    
    # Loop-based computation for dx
    for b in range(batch_size):
        for i in range(in_chan):
            for o in range(num_filters):
                for h in range(out_h):
                    for w in range(out_w):
                        h_start = h * sh
                        w_start = w * sw
                        dx_padded[b, i, h_start:h_start+fh, w_start:w_start+fw] += W_flipped[i, o] * dout[b, o, h, w]
    
    # Remove padding from dx if necessary
    if padding == 'same':
        pad_h_top = pad_h // 2
        pad_h_bottom = pad_h - pad_h_top
        pad_w_left = pad_w // 2
        pad_w_right = pad_w - pad_w_left
        dx = dx_padded[:, :, pad_h_top:x_padded.shape[2]-pad_h_bottom, pad_w_left:x_padded.shape[3]-pad_w_right]
    else:
        dx = dx_padded
    
    return dx, dW

DERIVATIVE FOR DECONVOLUTION:

In [10]:
def deconv_backward(dout, x, W, stride=(1,1), padding=(0,0), output_padding=(0,0)):
    """
    Vectorized backward pass for batch deconvolution (transposed convolution)
    
    Parameters:
    dout : np.ndarray [batch, out_channels, out_h, out_w] - Gradient from next layer
    x    : np.ndarray [batch, in_channels, h_in, w_in] - Input from forward pass
    W    : np.ndarray [out_channels, in_channels, fh, fw] - Filters
    stride, padding, output_padding: Same as forward pass
    
    Returns:
    dx : np.ndarray [batch, in_channels, h_in, w_in] - Gradient w.r.t input
    dW : np.ndarray [out_channels, in_channels, fh, fw] - Gradient w.r.t filters
    """
    # Extract dimensions
    batch_size, in_chan, h_in, w_in = x.shape
    out_chan, _, fh, fw = W.shape
    sh, sw = stride
    
    # 1. Compute dx (input gradient)
    # The gradient for deconv input is a convolution with transposed/flipped filters
    W_flipped = np.flip(W, axis=(2,3))  # Spatial filter rotation
    W_trans = W_flipped.transpose(1, 0, 2, 3)  # [in_chan, out_chan, fh, fw]
    
    # Calculate dx using regular convolution with proper padding
    # For strided deconv, we need to handle specific padding
    dx = batch_convolution(dout, W_trans, stride)  # Use stride 1 for gradient
    
    # 2. Compute dW (filter gradient)
    # For deconv filter gradients, we correlate dilated input with output gradient
    dW = np.zeros_like(W)
    
    # Process each sample in batch (can be vectorized with einsum for production)
    for b in range(batch_size):
        # Create dilated input by inserting zeros based on stride
        x_dilated = np.zeros((in_chan, (h_in-1)*sh + 1, (w_in-1)*sw + 1))
        x_dilated[:, ::sh, ::sw] = x[b]  # Fill strided positions with original values
        
        # Create spatial windows for each position in output gradient
        windows = sliding_window_view(x_dilated, (fh, fw), axis=(1,2))
        
        # Vectorized window extraction using einsum
        valid_h = min(dout.shape[2], windows.shape[1])
        valid_w = min(dout.shape[3], windows.shape[2])
        
        # Compute correlation between windows and output gradient
        for i in range(in_chan):
            for o in range(out_chan):
                for h in range(valid_h):
                    for w in range(valid_w):
                        # Accumulate gradient contributions
                        dW[o, i] += dout[b, o, h, w] * windows[i, h, w]
    
    return dx, dW

ERROR METRICS:

In [11]:
def hybrid_loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
    vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
    perceptual_model = tf.keras.Model(vgg.input, vgg.get_layer('block3_conv3').output)
    pl = tf.reduce_mean(tf.square(perceptual_model(y_true) - perceptual_model(y_pred)))
    return 0.7*pl + 0.3*mse

In [12]:
def square_error(IP,OP):
    error=(IP-OP)**2
    return error

MODEL:

In [13]:
dataset_path="D:\REQS\PROJECT\dataset\\train"

#Hypreparameters
epochs=10
back_spin=5
learning_rate=0.0001
e=0.00001
batch_size=32
image_size=[3,32,32]

In [14]:
#MODEL:2 -- THE BASE

s_all=[2,2]
#3x32x32
conv_f1=np.random.randn(8,3,4,4)* np.sqrt(2/(3*4*4))
bn_f1=np.random.randn(2,8)* np.sqrt(2/(3*4*4))
#8x15x15
conv_f2=np.random.randn(12,8,3,3)* np.sqrt(2/(3*4*4))
bn_f2=np.random.rand(2,12)* np.sqrt(2/(3*4*4))
#12x7x7
deconv_f3=np.random.randn(10,12,5,5)* np.sqrt(2/(3*4*4))
bn_f3=np.random.rand(2,10)* np.sqrt(2/(3*4*4))
#10x17x17
deconv_f4=np.random.randn(6,10,4,4)* np.sqrt(2/(3*4*4))
bn_f4=np.random.rand(2,6)* np.sqrt(2/(3*4*4))
#6x36x36
deconv_f5=np.random.randn(3,6,5,5)* np.sqrt(2/(3*4*4))
bn_f5=np.random.rand(2,3)* np.sqrt(2/(3*4*4))
#3x75x75
conv_f6=np.random.randn(6,3,5,5)* np.sqrt(2/(3*4*4))
bn_f6=np.random.rand(2,6)* np.sqrt(2/(3*4*4))
#6x36x36
conv_f7=np.random.randn(10,6,4,4)* np.sqrt(2/(3*4*4))
bn_f7=np.random.rand(2,10)* np.sqrt(2/(3*4*4))
#10x17x17
conv_f8=np.random.randn(12,10,5,5)* np.sqrt(2/(3*4*4))
bn_f8=np.random.rand(2,12)* np.sqrt(2/(3*4*4))
#12x7x7
deconv_f9=np.random.randn(8,12,3,3)* np.sqrt(2/(3*4*4))
bn_f9=np.random.rand(2,8)* np.sqrt(2/(3*4*4))
#8x15x15
deconv_f10=np.random.randn(3,8,4,4)* np.sqrt(2/(3*4*4))
bn_f10=np.random.rand(2,3)* np.sqrt(2/(3*4*4))
#3x32x32

In [15]:
#flow 1,2,3,4,5,6,7,8,9,10
def forward(IP):
    a1=batch_convolution(IP,conv_f1,(2,2),'valid')
    bn_a1=batch_norm(a1, bn_f1,0.00001)
    act_a1=relu(bn_a1)
    #print(act_a1.shape)
    a2=batch_convolution(act_a1,conv_f2,(2,2),'valid',)
    bn_a2=batch_norm(a2, bn_f2,0.00001)
    act_a2=relu(bn_a2)
    #print(act_a2.shape)
    a3=batch_deconvolution(act_a2,deconv_f3,(2,2),'valid',(0,0))
    bn_a3=batch_norm(a3, bn_f3,0.00001)
    act_a3=relu(bn_a3)
    #print(act_a3.shape)
    a4=batch_deconvolution(act_a3,deconv_f4,(2,2),'valid',(0,0))
    bn_a4=batch_norm(a4, bn_f4,0.00001)
    act_a4=relu(bn_a4)
    #print(act_a4.shape)
    a5=batch_deconvolution(act_a4,deconv_f5,(2,2),'valid',(0,0))
    bn_a5=batch_norm(a5, bn_f5,0.00001)
    act_a5=relu(bn_a5)
    #print(act_a5.shape)
    a6=batch_convolution(act_a5,conv_f6,(2,2),'valid')
    bn_a6=batch_norm(a6, bn_f6,0.00001)
    act_a6=relu(bn_a6)
    #print(act_a6.shape)
    a7=batch_convolution(act_a6,conv_f7,(2,2),'valid')
    bn_a7=batch_norm(a7, bn_f7,0.00001)
    act_a7=relu(bn_a7)
    #print(act_a7.shape)
    a8=batch_convolution(act_a7,conv_f8,(2,2),'valid')
    bn_a8=batch_norm(a8, bn_f8,0.00001)
    act_a8=relu(bn_a8)
    a9=batch_deconvolution(act_a8,deconv_f9,(2,2),'valid',(0,0))
    bn_a9=batch_norm(a9, bn_f9,0.00001)
    act_a9=relu(bn_a9)
    a10=batch_deconvolution(act_a9,deconv_f10,(2,2),'valid',(0,0))
    bn_a10=batch_norm(a10, bn_f10,0.00001)
    act_a10=relu(bn_a10)
    
    return act_a10, bn_a10, a10, act_a9, bn_a9, a9, act_a8, bn_a8, a8, act_a7, bn_a7, a7, act_a6, bn_a6, a6, act_a5, bn_a5, a5, act_a4, bn_a4, a4, act_a3, bn_a3, a3, act_a2, bn_a2, a2, act_a1, bn_a1, a1

In [16]:
def m1_backward(IP, layers, losse, saver):
    act_a10, bn_a10, a10, act_a9, bn_a9, a9, act_a8, bn_a8, a8, act_a7, bn_a7, a7, act_a6, bn_a6, a6, act_a5, bn_a5, a5, act_a4, bn_a4, a4, act_a3, bn_a3, a3, act_a2, bn_a2, a2, act_a1, bn_a1, a1=layers
    d_act_a10=losse
    d_bn_a10=relu_backward(d_act_a10)
    d_a10, d_bn_f10=batch_norm_backward(d_bn_a10, a10, bn_f10[0,:])
    d_act_a9, d_deconv_f10=deconv_backward(d_a10, act_a9, deconv_f10, (2,2))
    #print(d_act_a9.shape)
    d_bn_a9=relu_backward(d_act_a9)
    d_a9, d_bn_f9=batch_norm_backward(d_bn_a9, a9, bn_f9[0,:])
    d_act_a8, d_deconv_f9=deconv_backward(d_a9, act_a8, deconv_f9, (2,2))
    #print(d_act_a8.shape)
    d_bn_a8=relu_backward(d_act_a8)
    d_a8, d_bn_f8=batch_norm_backward(d_bn_a8, a8, bn_f8[0,:])
    d_act_a7, d_conv_f8=conv_backward(d_a8, act_a7, conv_f8, (2,2))
    #print(d_act_a7.shape)
    d_bn_a7=relu_backward(d_act_a7)
    d_a7, d_bn_f7=batch_norm_backward(d_bn_a7, a7, bn_f7[0,:])
    d_act_a6, d_conv_f7=conv_backward(d_a7, act_a6, conv_f7, (2,2))
    #print(d_act_a6.shape)
    d_bn_a6=relu_backward(d_act_a6)
    d_a6, d_bn_f6=batch_norm_backward(d_bn_a6, a6, bn_f6[0,:])
    d_act_a5, d_conv_f6=conv_backward(d_a6, act_a5, conv_f6, (2,2))
    #print(d_act_a5.shape)
    d_bn_a5=relu_backward(d_act_a5)
    d_a5, d_bn_f5=batch_norm_backward(d_bn_a5, a5, bn_f5[0,:])
    d_act_a4, d_deconv_f5=deconv_backward(d_a5, act_a4, deconv_f5, (2,2))
    #print(d_act_a4.shape)
    d_bn_a4=relu_backward(d_act_a4)
    d_a4, d_bn_f4=batch_norm_backward(d_bn_a4, a4, bn_f4[0,:])
    d_act_a3, d_deconv_f4=deconv_backward(d_a4, act_a3, deconv_f4, (2,2))
    #print(d_act_a3.shape)
    d_bn_a3=relu_backward(d_act_a3)
    d_a3, d_bn_f3=batch_norm_backward(d_bn_a3, a3, bn_f3[0,:])
    d_act_a2, d_deconv_f3=deconv_backward(d_a3, act_a2, deconv_f3, (2,2))
    #print(d_act_a2.shape)
    d_bn_a2=relu_backward(d_act_a2)
    d_a2, d_bn_f2=batch_norm_backward(d_bn_a2, a2, bn_f2[0,:])
    d_act_a1, d_conv_f2=conv_backward(d_a2, act_a1, conv_f2, (2,2))
    #print(d_act_a1.shape)
    d_bn_a1=relu_backward(d_act_a1)
    d_a1, d_bn_f1=batch_norm_backward(d_bn_a1, a1, bn_f1[0,:])
    d_IP, d_conv_f1=conv_backward(d_a1, IP, conv_f1, (2,2))
    saver=d_IP
    return d_conv_f1, d_bn_f1, d_conv_f2, d_bn_f2, d_deconv_f3, d_bn_f3, d_deconv_f4, d_bn_f4, d_deconv_f5, d_bn_f5, d_conv_f6, d_bn_f6, d_conv_f7, d_bn_f7, d_conv_f8, d_bn_f8, d_deconv_f9, d_bn_f9, d_deconv_f10, d_bn_f10

In [17]:
#Update Parameters
def update_params(diffs, lr, conv_f1, conv_f2, deconv_f3, deconv_f4, deconv_f5, conv_f6, conv_f7, conv_f8, deconv_f9, deconv_f10, bn_f1, bn_f2, bn_f3, bn_f4, bn_f5, bn_f6, bn_f7, bn_f8, bn_f9, bn_f10):
    d_conv_f1, d_bn_f1, d_conv_f2, d_bn_f2, d_deconv_f3, d_bn_f3, d_deconv_f4, d_bn_f4, d_deconv_f5, d_bn_f5, d_conv_f6, d_bn_f6, d_conv_f7, d_bn_f7, d_conv_f8, d_bn_f8, d_deconv_f9, d_bn_f9, d_deconv_f10, d_bn_f10=diffs
    conv_f1=conv_f1-lr*d_conv_f1
    bn_f1-=lr*d_bn_f1
    conv_f2-=lr*d_conv_f2
    bn_f2-=lr*d_bn_f2
    deconv_f3-=lr*d_deconv_f3
    bn_f3-=lr*d_bn_f3
    deconv_f4-=lr*d_deconv_f4
    bn_f4-=lr*d_bn_f4
    deconv_f5-=lr*d_deconv_f5
    bn_f5-=lr*d_bn_f5
    conv_f6-=lr*d_conv_f6
    bn_f6-=lr*d_bn_f6
    conv_f7-=lr*d_conv_f7
    bn_f7-=lr*d_bn_f7
    conv_f8-=lr*d_conv_f8
    bn_f8-=lr*d_bn_f8
    deconv_f9-=lr*d_deconv_f9
    bn_f9-=lr*d_bn_f9
    deconv_f10-=lr*d_deconv_f10
    bn_f10-=lr*d_bn_f10
    return

In [18]:
def load_image_paths():
    """Loads all image file paths from the dataset directory."""
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp')
    image_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) 
                    if f.endswith(supported_formats)]
    return image_paths
    
def load_image(image_path):
    """Loads an image, resizes it, and normalizes pixel values to the range [0, 1]."""
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not load image: {image_path}")
    #Eimage = cv2.resize(image, image_size)
    image = image.astype(np.float64) / 255.0  # Normalize to [0, 1]
    return image


In [19]:
def get_batch(image_paths):
    """Generates a batch of images for training."""
    batch_paths = np.random.choice(image_paths, batch_size, replace=False)
    images = np.array([load_image(path) for path in batch_paths])
    return images
    
image_paths = load_image_paths()
for epoch in range(epochs):
    total_loss = 0
    num_batches = len(image_paths) // batch_size
        
    for batch_index in range(num_batches):
        batch = get_batch(image_paths)
        batch = np.transpose(batch, (0,3,1,2))  # Proper channel-first format
        # Forward pass through the model
        results = forward(batch)
        act_a10, bn_a10, a10, act_a9, bn_a9, a9, act_a8, bn_a8, a8, act_a7, bn_a7, a7, act_a6, bn_a6, a6, act_a5, bn_a5, a5, act_a4, bn_a4, a4, act_a3, bn_a3, a3, act_a2, bn_a2, a2, act_a1, bn_a1, a1=results
            
        # Compute error (assume the error function is already implemented in the model)
        losse = square_error(act_a10, batch)
        loss =np.mean(losse)
        # Backward pass to compute gradients
        saver=np.zeros((batch_size, 3, 32, 32))
        m1_backward(batch, results, losse, saver)
        box=m1_backward(batch, results, 0.7*losse+0.3*saver, saver)
            
        # Update model parameters (assume the model has a method to update its parameters)
        update_params(box, learning_rate, conv_f1, conv_f2, deconv_f3, deconv_f4, deconv_f5, conv_f6, conv_f7, conv_f8, deconv_f9, deconv_f10, bn_f1, bn_f2, bn_f3, bn_f4, bn_f5, bn_f6, bn_f7, bn_f8, bn_f9, bn_f10)
        
        total_loss += loss

        if (batch_index + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_index+1}/{num_batches}], Loss: {loss:.4f}")
        
    avg_loss = total_loss / num_batches
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}")

Epoch [1/10], Batch [2/1562], Loss: 0.2507
Epoch [1/10], Batch [4/1562], Loss: 0.2791
Epoch [1/10], Batch [6/1562], Loss: 0.2813
Epoch [1/10], Batch [8/1562], Loss: 0.2691
Epoch [1/10], Batch [10/1562], Loss: 0.2601
Epoch [1/10], Batch [12/1562], Loss: 0.2785
Epoch [1/10], Batch [14/1562], Loss: 0.2557
Epoch [1/10], Batch [16/1562], Loss: 0.2673
Epoch [1/10], Batch [18/1562], Loss: 0.2794
Epoch [1/10], Batch [20/1562], Loss: 0.2854
Epoch [1/10], Batch [22/1562], Loss: 0.2906
Epoch [1/10], Batch [24/1562], Loss: 0.2850
Epoch [1/10], Batch [26/1562], Loss: 0.2978
Epoch [1/10], Batch [28/1562], Loss: 0.2700
Epoch [1/10], Batch [30/1562], Loss: 0.2456
Epoch [1/10], Batch [32/1562], Loss: 0.3333
Epoch [1/10], Batch [34/1562], Loss: 0.2752
Epoch [1/10], Batch [36/1562], Loss: 0.2754
Epoch [1/10], Batch [38/1562], Loss: 0.2836
Epoch [1/10], Batch [40/1562], Loss: 0.2828
Epoch [1/10], Batch [42/1562], Loss: 0.2632
Epoch [1/10], Batch [44/1562], Loss: 0.3082
Epoch [1/10], Batch [46/1562], Loss:

KeyboardInterrupt: 

In [20]:
params = [
    ('conv_f1', conv_f1), ('bn_f1', bn_f1),
    ('conv_f2', conv_f2), ('bn_f2', bn_f2),
    ('deconv_f3', deconv_f3), ('bn_f3', bn_f3),
    ('deconv_f4', deconv_f4), ('bn_f4', bn_f4),
    ('deconv_f5', deconv_f5), ('bn_f5', bn_f5),
    ('conv_f6', conv_f6), ('bn_f6', bn_f6),
    ('conv_f7', conv_f7), ('bn_f7', bn_f7),
    ('conv_f8', conv_f8), ('bn_f8', bn_f8),
    ('deconv_f9', deconv_f9), ('bn_f9', bn_f9),
    ('deconv_f10', deconv_f10), ('bn_f10', bn_f10)
]

print(",\n".join([f"{name}: {value}" for name, value in params]))


conv_f1: [[[[-1.56487065e-01 -3.33602432e-02  6.55505149e-02  8.22579009e-03]
   [ 2.82023589e-01  1.57353966e-01 -3.29183754e-01  3.62629816e-01]
   [-9.06306069e-02 -9.58973486e-02 -4.83614573e-02 -2.91955416e-02]
   [ 2.64833867e-01  3.78313362e-02  2.95560116e-01  6.59930674e-02]]

  [[ 4.05560968e-02  6.74192424e-03  3.95390084e-01 -2.53835535e-02]
   [ 1.92845213e-01  9.42720072e-02 -2.01598696e-01  8.75793298e-02]
   [ 1.72253869e-01 -2.71234340e-01  1.15172738e-01 -3.39707497e-02]
   [-1.26036767e-01  4.70974755e-01  8.68314356e-02  1.80315843e-01]]

  [[ 7.81799346e-02  4.83612631e-03  3.11161899e-01  1.97501822e-01]
   [-1.36358757e-02 -2.72115745e-01  7.83127126e-02  8.99591501e-02]
   [ 1.26314144e-01  2.79721637e-01  5.06254076e-02  9.76659963e-02]
   [-1.23369942e-01 -1.99470407e-02 -2.48780821e-01  7.64277680e-02]]]


 [[[ 1.11167119e-01 -2.74905510e-01 -2.47251102e-01  4.13097904e-02]
   [-1.71913528e-01  3.01663437e-02 -1.30622520e-01 -9.21819598e-02]
   [-1.69424257e-